In [1]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

# openai_client = OpenAI()

# def llm(user_prompt, instructions=None, model="gpt-4o-mini"):
#     messages = []

#     if instructions:
#         messages.append({
#             "role": "system",
#             "content": instructions
#         })

#     messages.append({
#         "role": "user",
#         "content": user_prompt
#     })

#     response = openai_client.responses.create(
#         model=model,
#         input=messages
#     )

#     return response.output_text

True

In [2]:
from src.github import GithubRepositoryDataReader

In [3]:
# https://github.com/DataTalksClub/datatalksclub.github.io/tree/main/_podcast

In [4]:
def read_github_data():
    allowed_extensions = {"md", "mdx"}

    repo_owner = 'DataTalksClub'
    repo_name = 'datatalksclub.github.io'

    reader = GithubRepositoryDataReader(
        repo_owner,
        repo_name,
        allowed_extensions=allowed_extensions
    )
    
    return reader.read()

In [ ]:
# Q4 
datatalsclub_data = read_github_data()
print(f"Downloaded {len(datatalsclub_data)} files")

Downloaded 765 files


In [6]:
datatalsclub_data[600].content

'---\nepisode: 2\nguests:\n- terezaiofciu\nids:\n  anchor: Decoding-Data-Science-Job-Descriptions---Tereza-Iofciu-e1m079l\n  youtube: bqxBiIwtmX4\nimage: images/podcast/s10e02-decoding-data-science-job-descriptions.jpg\nlinks:\n  anchor: https://anchor.fm/datatalksclub/episodes/Decoding-Data-Science-Job-Descriptions---Tereza-Iofciu-e1m079l\n  apple: https://podcasts.apple.com/us/podcast/decoding-data-science-job-descriptions-tereza-iofciu/id1541710331?i=1000575150345\n  spotify: https://open.spotify.com/episode/4v6h48B0c0Je8xLMo5zMs5?si=hcMUqpPPQYm2vrdi2py1UQ\n  youtube: https://www.youtube.com/watch?v=bqxBiIwtmX4\nseason: 10\nshort: Decoding Data Science Job Descriptions\ntitle: Decoding Data Science Job Descriptions\ntranscript:\n- line: This week, we\'ll talk about decoding the data science job description. We\n    have a special guest today, Tereza. Tereza is an experienced data practitioner.\n    She worked as a data science manager, a data scientist, data engineer, product\n    m

In [7]:
import frontmatter
import re
# from frontmatter.default_handlers import YAMLHandler, TOMLHandler


def safe_load_frontmatter(raw_text):
    # Add quotes around any {{ ... }} that isn't already quoted
    if "{{" in raw_text and "}}" in raw_text:
        # Add quotes around any {{ ... }} that isn't already quoted
        raw_text = re.sub(r'(?<!")({{.*?}})(?!")', r'"\1"', raw_text)
    
    # Now safely parse with frontmatter
    return frontmatter.loads(raw_text)


def parse_data(data_raw):

    data_parsed = []
    try:
        for f in data_raw:
            folder = f.filename.split('/')[0]

            if folder != '_podcast':
                continue 
            post = safe_load_frontmatter(f.content)
            data = post.to_dict()
            data['filename'] = f.filename
            data_parsed.append(data)
    except Exception as e:
        print(f"Error processing {f}: {e}")
        
    
    return data_parsed


In [8]:
parsed_data = parse_data(datatalsclub_data)

In [ ]:
# Q4 - 185 podcasts
len(parsed_data)

185

In [43]:
from typing import Any, Dict, Iterable, List

def join_transcript(transcript: List):
    content = ''
    for entry in transcript:
        content += entry['line'].replace('\n', ' ')

    return content

def sliding_window(
        seq: Iterable[Any],
        size: int,
        step: int
    ) -> List[Dict[str, Any]]:
    """
    Create overlapping chunks from a sequence using a sliding window approach.

    Args:
        seq: The input sequence (string or list) to be chunked.
        size (int): The size of each chunk/window.
        step (int): The step size between consecutive windows.

    Returns:
        list: A list of dictionaries, each containing:
            - 'start': The starting position of the chunk in the original sequence
            - 'content': The chunk content

    Raises:
        ValueError: If size or step are not positive integers.

    Example:
        >>> sliding_window("hello world", size=5, step=3)
        [{'start': 0, 'content': 'hello'}, {'start': 3, 'content': 'lo wo'}]
    """
    if size <= 0 or step <= 0:
        raise ValueError("size and step must be positive")

    seq_list = seq.split(' ')
    n = len(seq_list)
    result = []
    for i in range(0, n, step):
        batch = seq_list[i:i+size]
        batch_content = " ".join(batch)
        result.append({'transcript': batch_content})
        if i + size > n:
            break

    return result


def join_transcript(transcript: list):
    content = ''
    for entry in transcript:
        if 'header' in entry:
            content += entry['header'].replace('\n', ' ')
            content += '. '

        if 'line' in entry:
            content += entry['line'].replace('\n', ' ')
            content += '. '
    
    return content


def chunk_documents(
        documents: Iterable[Dict[str, str]],
        size: int = 30,
        step: int = 15,
        content_field_name: str = 'transcript'
) -> List[Dict[str, str]]:
    """
    Split a collection of documents into smaller chunks using sliding windows.

    Takes documents and breaks their content into overlapping chunks while preserving
    all other document metadata (filename, etc.) in each chunk.

    Args:
        documents: An iterable of document dictionaries. Each document must have a content field.
        size (int, optional): The maximum size of each chunk. Defaults to 2000.
        step (int, optional): The step size between chunks. Defaults to 1000.
        content_field_name (str, optional): The name of the field containing document content.
                                          Defaults to 'content'.

    Returns:
        list: A list of chunk dictionaries. Each chunk contains:
            - All original document fields except the content field
            - 'start': Starting position of the chunk in original content
            - 'content': The chunk content

    Example:
        >>> documents = [{'content': 'long text...', 'filename': 'doc.txt'}]
        >>> chunks = chunk_documents(documents, size=100, step=50)
        >>> # Or with custom content field:
        >>> documents = [{'text': 'long text...', 'filename': 'doc.txt'}]
        >>> chunks = chunk_documents(documents, content_field_name='text')
    """
    try:
        
        results = []
        for doc in documents:
            if content_field_name not in doc:
                doc[content_field_name] = ''
                results.extend(doc)
                continue
            
            podcast_copy = doc.copy()
            podcast_copy['guests'] = ",".join(podcast_copy['guests'])
            podcast_content = podcast_copy.pop(content_field_name)
            podcast_content = join_transcript(podcast_content)
            chunks = sliding_window(podcast_content, size=size, step=step)
            for chunk in chunks:
                chunk.update(podcast_copy)
            
            results.extend(chunks)

        return results
    
    except Exception as e:
        print(f"Unexpected Error: {e}")

In [44]:
# Q5 - Chunk the lines 
# 

In [45]:

chunks = chunk_documents(parsed_data)

In [50]:
len(chunks)

94539

In [47]:
chunks[8]

{'transcript': "for some time. I'm really happy to be here.. Jekaterina’s background. I tried to invite you multiple times. Finally, we managed to do this. [chuckles] Before we start with our",
 'episode': 8,
 'guests': 'jekaterinakokatjuhha',
 'ids': {'anchor': 'The-Journey-of-a-Data-Generalist-From-Bioinformatics-to-Freelancing---Jekaterina-Kokatjuhha-e1upvim',
  'youtube': 'FRi0SUtxdMw'},
 'image': 'images/podcast/s12e08-journey-of-data-generalist-from-bioinformatics-to-freelancing.jpg',
 'links': {'anchor': 'https://anchor.fm/datatalksclub/episodes/The-Journey-of-a-Data-Generalist-From-Bioinformatics-to-Freelancing---Jekaterina-Kokatjuhha-e1upvim',
  'apple': 'https://podcasts.apple.com/us/podcast/the-journey-of-a-data-generalist-from/id1541710331?i=1000599125044',
  'spotify': 'https://open.spotify.com/episode/5fB185hGlGYQmdk0kbIsPv?si=YtnsaYNzTc-fl7emZ2IjEA',
  'youtube': 'https://www.youtube.com/watch?v=FRi0SUtxdMw'},
 'season': 12,
 'short': 'The Journey of a Data Generalist: F

In [48]:
from minsearch import Index

index = Index(
    text_fields=["transcript", "title", "guests"]
)

index.fit(chunks)

In [49]:
# Q6 
# episode 7
search_results = index.search('how do I make money with AI?')
search_results[0]

{'transcript': "to do anything if you're not familiar with AI.. Does that mean that if I'm a beginner, I cannot take part in Omdena challenges?. How to take part in AI",
 'episode': 7,
 'guests': 'saraelateif',
 'ids': {'anchor': 'atatalksclub/episodes/Make-an-Impact-Through-Volunteering-Open-Source-Work---Sara-EL-ATEIF-e2g4dan',
  'youtube': 'aHdaIwOEI8Q'},
 'image': 'images/podcast/s17e07-make-impact-through-volunteering-open-source-work.jpg',
 'links': {'anchor': 'https://podcasters.spotify.com/pod/show/datatalksclub/episodes/Make-an-Impact-Through-Volunteering-Open-Source-Work---Sara-EL-ATEIF-e2g4dan',
  'apple': 'https://podcasts.apple.com/us/podcast/make-an-impact-through-volunteering-open-source-work/id1541710331?i=1000646627892',
  'spotify': 'https://open.spotify.com/episode/7tZSSgv1yAlnoMyB4ggQmb?si=AqDaME2QS26usoZjOEWNtQ',
  'youtube': 'https://www.youtube.com/watch?v=aHdaIwOEI8Q'},
 'season': 17,
 'short': 'Make an Impact Through Volunteering Open Source Work',
 'title': 'M

In [42]:
def search(query):
    return index.search(
        query=query,
        num_results=5
    )


search('how do I make money with AI?')

[{'transcript': "I think nowadays, that's one of the main things you have to do as a company or as a founder – you must really prove, “Okay, we have the possibility to make money and not just a good idea. We don't need 100 million to get the first money. We also have good business cases already here.” But honestly, [cross-talk] that's a tough way. I mean, you need to find first customers who are willing to pay money, even if you're probably not ready with everything. But it's an operational problem you can definitely solve.. How do you earn money",
  'episode': 8,
  'guests': 'mariabruckert',
  'ids': {'anchor': 'atatalksclub/episodes/AI-for-Digital-Health---Maria-Bruckert-e2cejoc',
   'youtube': 'whpkDmVVGUE'},
  'image': 'images/podcast/s16e08-ai-for-digital-health.jpg',
  'links': {'anchor': 'https://podcasters.spotify.com/pod/show/datatalksclub/episodes/AI-for-Digital-Health---Maria-Bruckert-e2cejoc',
   'apple': 'https://podcasts.apple.com/us/podcast/ai-for-digital-health-maria-br